In [ ]:
import seaborn as sns

from scipy.stats import kurtosis, skew
import pandas as pd
from rcv_distribution import *
from rcv_dimensionality import *
from voting_rules import *
import pandas as pd
import os 

In [1]:
winners = pd.DataFrame(columns=["filename", "diff", "IRV1", "Condorcet1", "Plurality1", "Approval1" "IRV2", "Condorcet2", "Plurality2", "Approval2"])
print(winners)
#winners.columns = ["filename", "IRV1", "Condorcet1"]
directory = "dataverse_files"
for filename in os.listdir(directory):
    try:
        diff = []
        #winners["filename"].index[0] = filename
        print(filename)
        csv = os.path.join(directory, filename)
        ballots, candidates = parse_election_data(csv)
        #frequency, first = freq(ballots, candidates)
        
        # Perform the RCV analysis
        test = perform_rcv_analysis(csv, n_runs=1000)
        mds_1d_coordinates, mds_2d_coordinates, most_common_order, order_frequencies, candidate_names = test

        # Print the normalized distances between candidates and plot the MDS analysis
        normalized_distances = get_distances_normalized(most_common_order, mds_1d_coordinates, candidate_names)

        election = voting_rules(ballots, candidates)
        irv1 = election.irv()
        condorcet1 = election.condorcet()
        plurality1 = election.plurality()
        approval1 = election.approval()
        
        #print({'filename': filename, 'IRV1': irv_winner, 'Condorcet1': condorcet_winner})
        #print(winners)
        consistent_ballots = {}
        most_consistent_permutation = []
        permutation_numbers = []
        for candidate in normalized_distances:
            most_consistent_permutation.append(candidate)
            permutation_numbers.append(normalized_distances[candidate])

        points = {}

        equal = {}
        for i in range(len(most_consistent_permutation)):
            equal[most_consistent_permutation[i]] = i

        for ballot in ballots:
            equal_distances = []
            for candidate in ballot:
                equal_distances.append(equal[candidate])

            mds_distances = []
            for candidate in ballot:
                mds_distances.append(normalized_distances[candidate])

            #checking if the ballot is consistent with the mds permutation but assuming they are equaly distances 
            check_consistency = evaluate_ballot_consistency(equal_distances, len(candidates))
            if check_consistency[0] is True:
                consistent_ballots[ballot] = ballots[ballot]

        election = voting_rules(consistent_ballots, candidates)
        irv2 = election.irv()
        condorcet2 = election.condorcet()
        plurality2 = election.plurality()
        approval2 = election.approval()
        if (irv1 != irv2):
            diff.append("IRV")
        if (condorcet1 != condorcet2):
            diff.append("Condorcet")
        if (plurality1 != plurality2):
            diff.append("Plurality")
        if (approval1 != approval2):
            diff.append("Approval")
        winners = winners.append({'filename': filename, 'diff': diff, 'IRV1': irv1, 'Condorcet1': condorcet1,
                                'Plurality1': plurality1, 'Approval1': approval1,
                                'IRV2': irv2, "Condorcet2": condorcet2, 
                                'Plurality2':plurality2, 'Approval2': approval2}, ignore_index=True)
        #print({'filename': filename, 'IRV1': irv_winner, 'Condorcet1': condorcet_winner, 'IRV2': irv2, "Condorcet2": condorcet2})
    except:
        print(filename, " threw an error")
winners.to_csv('interval_winners.csv', index=False)


NameError: name 'pd' is not defined

In [29]:
print(winners)

                                     filename               IRV1  \
0         Alaska_11082022_SenateDistrictR.csv      Bishop, Click   
1         Alaska_11082022_SenateDistrictS.csv  Hoffman, Lyman F.   
2  Berkeley_11062012_CityCouncilDistrict5.csv   LAURIE CAPITELLI   
3  Berkeley_11062012_CityCouncilDistrict6.csv      SUSAN WENGRAF   

          Condorcet1                     IRV2               Condorcet2  
0      Bishop, Click  Williams, Robert "Bert"  Williams, Robert "Bert"  
1  Hoffman, Lyman F.            Keppel, Willy            Keppel, Willy  
2   LAURIE CAPITELLI              SOPHIE HAHN              SOPHIE HAHN  
3      SUSAN WENGRAF            SUSAN WENGRAF            SUSAN WENGRAF  


In [31]:
winners.to_csv('winners.csv', index=False)